In [1]:
from pathlib import Path
import os

from astropy.nddata import CCDData
from astropy.visualization import hist
from astropy.stats import mad_std

import ccdproc as ccdp
import matplotlib.pyplot as plt
import numpy as np
import sys
from astropy import units as u


sys.path.append("/home/idies/workspace/Storage/madisonleblanc/persistent/hrpo-pipeline")
import fpnfix
from aperturePhot import showimage as show_image

In [2]:
path = "/home/idies/workspace/Storage/madisonleblanc/persistent/reductions_ASTR4261/"
day = "20200916"
path_day = os.path.join(path, day)

data_red = Path(path_day, "Reduced_Images")
data_red.mkdir(exist_ok = True)

In [3]:
files = ccdp.ImageFileCollection(path_day)
files.summary['file', 'imagetyp', 'filter', 'exptime', 'naxis1', 'naxis2']

file,imagetyp,filter,exptime,naxis1,naxis2
str18,str11,object,float64,int64,int64
Bias-0011x1.fit,Bias Frame,--,0.0,1024,1024
Bias-0012x2.fit,Bias Frame,--,0.0,512,512
Bias-0021x1.fit,Bias Frame,--,0.0,1024,1024
Bias-0022x2.fit,Bias Frame,--,0.0,512,512
Bias-0031x1.fit,Bias Frame,--,0.0,1024,1024
Bias-0032x2.fit,Bias Frame,--,0.0,512,512
Bias-0041x1.fit,Bias Frame,--,0.0,1024,1024
Bias-0042x2.fit,Bias Frame,--,0.0,512,512
Bias-0051x1.fit,Bias Frame,--,0.0,1024,1024


In [4]:
calibrated_biases = files.files_filtered(imagetyp='Bias Frame', include_path=True)

combined_bias = ccdp.combine(calibrated_biases,
                             method='median',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std, unit = "adu")

combined_bias.meta['combined'] = True

combined_bias.write(os.path.join(data_red / 'combined_bias.fit'), overwrite = True)

TypeError: CCDData objects are not the same size.

In [ ]:
calibrated_darks = files.files_filtered(imagetyp='Dark Frame', include_path=True)
reduced_darks= []
exptime= []

for frame in calibrated_darks:
    image = ccdp.CCDData.read(frame, unit = 'adu')
    exptime.append(image.header["exposure"])
    reduced_darks.append(ccdp.subtract_bias(image, combined_bias))
    
combined_darks = ccdp.combine(reduced_darks,
                             method='median',
                             sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                             sigma_clip_func=np.ma.median, sigma_clip_dev_func=mad_std, unit = "adu", scale =1 / np.array(exptime))

combined_darks.header["exposure"] = 1
combined_darks.header['exptime'] = 1

combined_darks.meta['combined'] = True

combined_darks.write(os.path.join(data_red / 'combined_darks.fit'), overwrite = True)

In [ ]:
import fpnfix
import ccdproc
import aperturePhot as ap


os.chdir(path_day)
combfpn = fpnfix.buildFPNFrame(files, redpath = "./Reduced_Images", darkim = "combined_darks.fit") #Note this assumes that the SO Darks were taken as 'Light Frame' 
                                       #with OBJECT='SODark' - use imagetyp and object keyword arguments 
                                       #to change this if needed
                                       
#Show the combined FPN frame build from 60 second SO Darks               
ap.showimage(combfpn[60])

In [ ]:
flat_imagetyp = 'Flat Field'

flat_filters = set(h['filter'] for h in files.headers(imagetyp=flat_imagetyp))
flat_filters

In [5]:
scale = lambda a: 1/np.median(a)

master_flats = {}

for filt in flat_filters:
    reduced_flats = []
    calibrated_flats = files.files_filtered(imagetyp = flat_imagetyp, filter = filt, include_path=True)
    
    for flat in calibrated_flats:
        image = ccdp.CCDData.read(flat, unit = "adu")
        image = ccdp.subtract_bias(image, combined_bias)
        image = ccdp.subtract_dark(image, combined_darks, scale=True, exposure_unit=u.s, exposure_time = "exptime")
        #image = fpnfix.subtract_fpn(image,combfpn[60],mesh_size=32, verbose = 2)
        reduced_flats.append(image)
    
    combined_flats = ccdp.combine(reduced_flats,
                                 method='median',
                                 sigma_clip=True, sigma_clip_low_thresh=5, sigma_clip_high_thresh=5,
                                 sigma_clip_func=np.ma.median, signma_clip_dev_func=mad_std, unit = "adu", scale = scale
                                )

    master_flats[filt]=combined_flats.copy()

    combined_flats.write(data_red / (filt.replace("","_")+"_combined_flats.fit"), overwrite = True)

NameError: name 'flat_filters' is not defined

In [ ]:
show_image(combined_bias)

In [ ]:
show_image(combined_darks)

In [ ]:
for filt,image in master_flats.items():
    show_image(image)

In [ ]:
import ccdproc
images = files.files_filtered(imagetyp="Light Frame", include_path = True)

for image in images:
    var = ccdp.CCDData.read(image, unit = "adu")
    var_filt = var.header["filter"]
    
    corr_filt = master_flats[var_filt]
    
    var = ccdproc.ccd_process(var,
                             master_bias = combined_bias, dark_frame = combined_darks,
                             master_flat = corr_filt, exposure_key="exposure", exposure_unit = u.second,
                             dark_scale = True)
    proc_data = os.path.basename(image)
    var.write(data_red / ("BDF_"+proc_data), overwrite = True)

In [ ]:
import plateSolve
pathlist = Path(data_red).glob('**/*V.fit')

for path in pathlist:
    Vimage = ccdp.CCDData.read(path, unit="adu")
    show_image(Vimage)
    
    check = plateSolve.autoSolve(path,Vimage)
    print("Check should be true if autoSolve worked correctly.")
    print(path,check)
    print(check)

In [ ]:
import plateSolve
pathlist = Path(data_red).glob('**/*V.fit')

for path in pathlist:
    Vimage = ccdp.CCDData.read(path, unit="adu")
    show_image(Vimage)
    
    check = plateSolve.autoSolve(path,Vimage)
    print("Check should be true if autoSolve worked correctly.")
    print(path,check)
    print(check)

In [ ]:
import plateSolve
pathlist = Path(data_red).glob('**/*V.fit')

for path in pathlist:
    Vimage = ccdp.CCDData.read(path, unit="adu")
    show_image(Vimage)
    
    check = plateSolve.autoSolve(path,Vimage)
    print("Check should be true if autoSolve worked correctly.")
    print(path,check)
    print(check)

In [ ]:
import plateSolve

check = plateSolve.autoSolve(R,Rimage)
print("Check should be true if autoSolve worked correctly.")
print(R,check)
print(check)

In [ ]:
checkV = plateSolve.autoSolve(V,Vimage)
print("Check should be true if autoSolve worked correctly.")
print(V,checkV)
print(checkV)

In [ ]:
plateSolve.manualSolvePlot(V,Vimage)

In [ ]:
gaia_list = [0,1, 3, 9, 12,17,22,24,54]
image_list = [0,1,4,5,8, 11, 13, 15,16]

check = plateSolve.manualSolve(V,Vimage,gaia_list,image_list)
print("Check is True if successful")
print(V,check)